# Optimisation and its Pitfalls

In the context of PKPD modelling numerical optimisation is often used to find estimates for model parameters by minimising an objective function $L$ that quantifies the distance of the model predictions to the measured data. In this notebook we discuss some of the common reasons why numerical optimisation may fail to find the best parameter set, and more importantly may fail to consistently produce estimates over multiple optimisation runs.

This notebook is complementary to the identifiability notebooks provided in this repository [1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb).

## Toy model: Tumour growth in absence of treatment

For clarity let us illustrate the pitfalls of numerical optimisers for a specific example. In [1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb) we introduced a structural model for the tumour growth in absence of treatment in which the tumour grows exponentially for tumour volumes $V^s_T$ below a critical volume $V_{\text{crit}}$, and linearly above the critical tumour volume

\begin{equation*}
    \frac{\text{d}V^s_T}{\text{d}t} = \frac{\lambda V^s_T}{V^s_T / V_{\text{crit}} + 1}.
\end{equation*}

Here, $\lambda $ is the exponential growth rate. We can see that for large tumour volumes $V^s_T\gg V_{\text{crit}}$ the rate of change of the tumour will assume a constant growth rate $\lambda V_{\text{crit}}$, i.e. the tumour grows linearly for large tumour volumes.

We generate a dataset by forward simulating the model for the structural model parameters

\begin{equation*}
    \psi = (V_0, V_{\text{crit}}, \lambda ) = (0.14\, \text{cm}^3, 0.24\, \text{cm}^3, 0.55\, 1/\text{day}),
\end{equation*}

which we found in [1](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb) for mouse ID 40. We will record the tumour volume noise-free every second day for a period of 30 days to generate a dataset similar to the experimental observations.

The toy inverse problem is completed by defining the Squared Distance Error Measure as our objective function

\begin{equation*}
    L(\psi | V^\text{obs}_{T}) = \sum ^{n}_{i=1}\left( V^\text{obs}_{T, i} - V^s_T(t_i, \psi)\right) ^2,
\end{equation*}

where $n$ is the number of simulated measurements, $V^\text{obs}_{T, i}$ is the "measured" tumour volume at time $t_i$ and $V^s_T(t_i, \psi)$ is the model prediction at time $t_i$ for model parameters $\psi $.

## 1. Limiting the search space

A generic problem of numerical optimisation routines is that there is only a finite window of parameter values that can be feasibly explored. Outside this window evaluation of the objective function will have significant numerical erros.

Let us begin by showing objecive function over range of many parameters:

We use dimensionless model with log-transformed parameters as in 1.

$v_0 = 0.14$, $a_0 = 0.24$, $a_1 = 0.24 * 0.55$

In [17]:
#
# Define pints model wrapper such that myokit model can be used for inference.
#

import myokit
import pints

from pkpd import model as m


# Wrap myokit model, so it can be used with pints
class PintsModel(pints.ForwardModel):
    def __init__(self):
        # Create myokit model
        model = m.create_dimless_tumour_growth_model()

        # Create simulator
        self.sim = myokit.Simulation(model)

    def n_parameters(self):
        """
        Number of parameters to fit. Here log v, log a_0, log a_1.
        """
        return 3

    def n_outputs(self):
        return 1

    def simulate(self, log_parameters, times):
        # Reset simulation
        self.sim.reset()

        # Sort input parameters
        initial_volume, a_0, a_1 = np.exp(log_parameters)

        # Set initial condition
        self.sim.set_state([initial_volume])

        # Set growth constants
        self.sim.set_constant('central.a_0', a_0)
        self.sim.set_constant('central.a_1', a_1)

        # Define logged variable
        loggedVariable = 'central.volume_t'

        # Simulate
        output = self.sim.run(times[-1] + 1, log=[loggedVariable], log_times=times)
        result = output[loggedVariable]

        return np.array(result)

In [23]:
#
# Simulate "noise-free" data for with v_0 = 0.14, a_0 = 0.24, a_1= 0.24 * 0.55. 
#
# This cell needs the above defined model.
# [PintsModel]
#

import numpy as np


# Define characteristic scales
characteristic_volume = 1  # in cm^3
characteristic_time = 1  # in day

# Define simulation times in day
data_times = np.linspace(start=0, stop=30, num=15)

# Define model parameters v_0, a_0, a_1
true_log_parameters = np.log([0.14, 0.24, 0.24 * 0.55])

# Instantiate model
model = PintsModel()

# Define dimensionless times
dimless_times = data_times / characteristic_time

# Simulate data
dimless_volumes = model.simulate(true_log_parameters, dimless_times)
data_volumes = dimless_volumes * characteristic_volume


In [19]:
#
# Run optimisation multiple times with and without boundaries
#
# This cell needs the above defined wrapped myokit model and the simulated data, as well as the characteristic time and volume scale:
# [PintsModel, simulated_data, characteristic_time, characteristic_volume]
#

import myokit
import numpy as np
import pandas as pd
import pints


# Define number of different optimisation settings
# [bounded search space, open search space]
n_opt_types = 2

# Define number of optimisation runs for each individual
n_runs = 10

# Define container for the structural model estimates
# Shape (n_opt_types, n_runs, n_parameters)
n_parameters = 3
recovered_parameters = np.empty(shape=(n_opt_types, n_runs, n_parameters))

# Define container for the objective function score for the optimised parameters
recovered_parameters_scores = np.empty(shape=(n_opt_types, n_runs))

# Define random starting points over many orders of magnitude
# Shape = (n_runs, n_parameters)
initial_parameters = np.random.uniform(low=1E-3, high=1E3, size=(n_runs, n_parameters))

# Find mouse parameters
for index in range(n_opt_types):
    # Get relevant time points
    times = data_times / characteristic_time

    # Get measured tumour volumes
    observed_volumes = data_volumes / characteristic_volume

    # Create inverse problem
    problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

    # Create sum of squares error objective function
    error = pints.SumOfSquaresError(problem)

    # Create boundaries to biologically relevant values
    boundary = pints.RectangularBoundaries(lower=np.log([1E-3]*3), upper=np.log([1E3]*3))

    # Run optimisation multiple times
    for run_id, initial_params in enumerate(initial_parameters):
        # Transform parameters to log-scale
        log_initial_params = np.log(initial_params)

        # Create optimisation controller with a CMA-ES optimiser
        if index == 0:
            # Create optimiser with bounded search space
            optimiser = pints.OptimisationController(
                function=error,
                x0=log_initial_params,
                boundaries=boundary,
                method=pints.CMAES)
        if index == 1:
            # Create optimiser with bounded search space
            optimiser = pints.OptimisationController(
                function=error,
                x0=log_initial_params,
                method=pints.CMAES)

        # Disable logging mode
        optimiser.set_log_to_screen(False)

        # Parallelise optimisation
        optimiser.set_parallel(True)

        # Find optimal parameters
        try:
            estimates, score = optimiser.run()
        except:
            # If inference breaks fill estimates with nan
            estimates = np.array([np.nan, np.nan, np.nan])
            score = np.nan

        # Save estimates and score (back transformed to linear scale)
        recovered_parameters[index, run_id, :] = np.exp(estimates)
        recovered_parameters_scores[index, run_id] = score


----------------------------------------
Unexpected termination.
Current best score: 10.156246398491989
Current best position:
 4.51990710707117271e-01
 6.82247804558307848e+02
 6.78276907497045613e+02
----------------------------------------

----------------------------------------
Unexpected termination.
Current best score: 2.607533564388874
Current best position:
-9.46999195455406062e-02
 5.96300708013394569e+02
 5.93069773285688484e+02
----------------------------------------


In [20]:
#
# Transform dimensionless parameters to new set of biological parameters.
# 
# This cell needs the above inferred dimensionless parameters and the characteristic volume and time scale:
# [recovered_parameters, characteristic_volume, characteristic_time]
#

import numpy as np


# Initialise container for backtransformed paramters
# Shape (n_mice, n_runs, n_parameters)
parameters = np.empty(shape=recovered_parameters.shape)

# Transform initial volumes
parameters[:, :, 0] = recovered_parameters[:, :, 0] * characteristic_volume

# Transform critical volume 
# V_crit = a_0 * V^c
parameters[:, :, 1] = recovered_parameters[:, :, 1] * characteristic_volume

# Transform exponential growth rate
# lambda = a_1 / a_0 / t^c
parameters[:, :, 2] = \
    recovered_parameters[:, :, 2] / recovered_parameters[:, :, 1] / characteristic_time


In [55]:
#
# Visualisation of the spread of optimised model parameters for multiple runs from different initial points.
#
# This cell needs the above optimised initial parameter from psi_0=(1, 1, 1) and the five runs from a random initial starting point, and their respective objective function scores, as well as the data
# [parameters, recovered_parameters_scores]
#

import plotly.colors
import plotly.graph_objects as go


# Get number of optimisation types
n_opt_types = parameters.shape[0]

# Define colorscheme
colors = plotly.colors.qualitative.Plotly[:n_opt_types]

# Get optimised parameters
optimised_parameters = parameters

# Get optimised parameters
scores = recovered_parameters_scores

# Create figure
fig = go.Figure()

# Box plot of optimised model parameters
for index in range(n_opt_types):
    # Get optimised parameters
    params = optimised_parameters[index, ...]

    # Get scores
    score = scores[index, :]

    # Label
    label = "Limited search space" if index == 0 else "Unlimited search space"

    # Create box plot of for initial tumour volume
    fig.add_trace(
        go.Box(
            y=params[:, 0],  
            name=label,
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[index],
            line_color=colors[index]))

    # Create box plot for critical volume
    fig.add_trace(
        go.Box(
            y=params[:, 1],  
            name=label,
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[index],
            line_color=colors[index]))

    # Create box plot of for exponential tumour growth
    fig.add_trace(
        go.Box(
            y=params[:, 2],  
            name=label,
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=False,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[index],
            line_color=colors[index]))
    
    # Create box plot of for objective function score
    fig.add_trace(
        go.Box(
            y=score,  
            name="Score: " + label,
            boxpoints='all',
            jitter=0.2,
            pointpos=-1.5,
            visible=True,
            marker=dict(
                symbol='circle',
                opacity=0.7,
                line=dict(color='black', width=1)),
            marker_color=colors[index],
            line_color=colors[index]))

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Add switch between mice
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[
                        {"visible": [True, False, False, True]*2},
                        {"yaxis": {"title": r'$\text{Initial tumour volume in cm}^3$'}}],
                    label="Initial tumour volume",
                    method="update"
                ),
                dict(
                    args=[
                        {"visible": [False, True, False, True]*2},
                        {"yaxis": {"title": r'$\text{Critical tumour volume in cm}^3$'}}],
                    label="Critical tumour volume",
                    method="update"
                ),
                dict(
                    args=[
                        {"visible": [False, False, True, True]*2},
                        {"yaxis": {"title": r'$\text{Growth rate in 1/day}$'}}],
                    label="Growth rate",
                    method="update"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()


In [51]:
fig['layout']["shapes"][0]

layout.Shape({
    'line': {'color': 'DarkGrey', 'dash': 'dash'},
    'name': 'Generating parameter value',
    'opacity': 0.5,
    'type': 'line',
    'x0': 0,
    'x1': 1,
    'xref': 'paper',
    'y0': 0.14,
    'y1': 0.14,
    'yref': 'y'
})

**Figure 1:** Limited versus unlimited search space. Initialised over $[10^{-3}, 10^3]$

Let us now try to understand why the optimisation fails by looking at the objective function over a significant parameter range

In [ ]:
#
# Evaluate objective function over many orders of magnitude.
#
# This cell needs the above simulated data and the generating set of model parameters
# [data_times, data_volumes]
#

#
# Run optimisation multiple times with and without boundaries
#
# This cell needs the above defined wrapped myokit model and the simulated data, as well as the characteristic time and volume scale:
# [PintsModel, simulated_data, characteristic_time, characteristic_volume]
#

import myokit
import numpy as np
import pandas as pd
import pints


# Define range of model parameters
v_0_range = np.

# Convert data times to dimensionless time
times = data_times / characteristic_time

# Convert data volumes to dimensionless volumes
observed_volumes = data_volumes / characteristic_volume

# Create inverse problem
problem = pints.SingleOutputProblem(PintsModel(), times, observed_volumes)

# Create sum of squares error objective function
error = pints.SumOfSquaresError(problem)

## Notebook references

- <a name="ref1"> [1] </a> [Identifiability of the Structural Model for Lung Cancer Growth in Absence of Treatment](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability.ipynb)